In [1]:
from flair.data import Sentence
from flair.models import TextClassifier
import pandas as pd

In [2]:
model = TextClassifier.load("models/inference_model_big/model_epoch_9.pt")

In [3]:
df = pd.read_csv("data/train_gpt.csv", sep='\t', index_col=[0])

In [4]:
df.head(2)

,date,money,text,gpt_class
0,07.11.2024,40500.00,За тур.поездку по договору №001 от 27.01.2023г,SERVICE
1,07.11.2024,"32600,00",За оказание услуг по договору №53Б-02746 от 23...,SERVICE


In [5]:
texts = [Sentence(text) for text in df["text"]]

In [6]:
len(texts)

4068

In [7]:
model.predict(texts, mini_batch_size=16, verbose=True)

Batch inference: 100%|██████████| 255/255 [00:24<00:00, 10.58it/s]


In [8]:
tags = [text.tag for text in texts]

In [9]:
df["bert_class"] = tags

In [10]:
df["final_class"] = [None]*len(df)

In [11]:
df['bert_class'].value_counts()

NON_FOOD_GOODS    1105
FOOD_GOODS         911
SERVICE            749
LOAN               390
LEASING            376
REALE_STATE        225
BANK_SERVICE       204
NOT_CLASSIFIED      92
TAX                 16
Name: bert_class, dtype: int64

In [12]:
df["gpt_class"].value_counts()

FOOD_GOODS        916
NON_FOOD_GOODS    863
SERVICE           859
LOAN              381
LEASING           380
REAL_ESTATE       309
BANK_SERVICE      211
NOT_CLASSIFIED    126
TAX                23
Name: gpt_class, dtype: int64

In [13]:
df["gpt_class"] = df["gpt_class"].replace("REAL_ESTATE", "REALE_STATE")

In [15]:
df.to_csv("data/train_gpt_bert.csv", sep='\t')

In [21]:
df[df["gpt_class"] != df["bert_class"]].to_csv("missmatch.csv", sep=',')

In [22]:
df.head(2)

,date,money,text,gpt_class,bert_class,final_class
0,07.11.2024,40500.00,За тур.поездку по договору №001 от 27.01.2023г,SERVICE,SERVICE,None
1,07.11.2024,"32600,00",За оказание услуг по договору №53Б-02746 от 23...,SERVICE,SERVICE,None


### Concatenate results

In [2]:
import numpy as np
import pandas as pd
import csv

In [3]:
df = pd.read_csv("data/train_gpt_bert.csv", sep='\t', index_col=[0])

In [4]:
df_miss = pd.read_csv("missmatch_.csv", index_col=[0])

In [5]:
df_miss[df_miss["final_class"].isna()]

,date,money,text,gpt_class,bert_class,final_class


In [6]:
set(df_miss['final_class'].to_list())

{'BANK_SERVICE',
 'FOOD_GOODS',
 'LEASING',
 'LOAN',
 'NON_FOOD_GOODS',
 'NOT_CLASSIFIED',
 'REALE_STATE',
 'SERVICE',
 'TAX'}

In [7]:
df_res = pd.concat([df.drop(columns=["final_class"]), df_miss[["final_class"]]], axis=1)

In [8]:
from typing import Optional


def make_class(bert_class: str, final_class: Optional[str]):
    if isinstance(final_class, str):
        return final_class
    else:
        return bert_class

In [9]:
df_res["class"] = df_res.apply(lambda x: make_class(x.bert_class, x.final_class), axis=1)

In [10]:
df_res[18:22]

,date,money,text,gpt_class,bert_class,final_class,class
18,07.11.2024,1990.00,Оплата коммунальных услуг,SERVICE,SERVICE,NaN,SERVICE
19,07.11.2024,4880000.00,Окончательный расчет по акту приема-передачи №...,LEASING,LEASING,NaN,LEASING
20,07.11.2024,1960000-00,Оплата аванса за ГСМ по счету А-0631 от 21.01....,NON_FOOD_GOODS,NON_FOOD_GOODS,NaN,NON_FOOD_GOODS
21,07.11.2024,24500-00,Оплата за программирование блока фары Mercedes...,SERVICE,NON_FOOD_GOODS,SERVICE,SERVICE


In [11]:
df_res = df_res.drop(columns=["date", "money", "gpt_class", "bert_class", "final_class"])

In [12]:
df_res.head()

,text,class
0,За тур.поездку по договору №001 от 27.01.2023г,SERVICE
1,За оказание услуг по договору №53Б-02746 от 23...,SERVICE
2,Оплата штрафа,TAX
3,Лечение по договору №Д-00359/24 от 08.03.2025,SERVICE
4,Оплата основного долга за период с 16.12.2024г...,BANK_SERVICE


### generated data

In [28]:
df_dev_json = {
"SERVICE": [
    "Оплата за ремонт системы охлаждения по договору №1029R-066 от 15 июля 2024.",
    "Услуги по техническому обслуживанию компьютеров по з/н №77-00991 за период октябрь 2024.",
    "Оплата за проведение корпоративного тренинга по счету №2024К-345 от 30.09.2024.",
    "За услуги уборки офиса по адресу: ул. Ленина, д. 4, по договору №314 от 01 марта 2024.",
    "Оплата за разработку программного обеспечения по лицензионному договору №6745-ПО от 12 июня 2024.",
    "Услуги по дизайну интерьера по договору №45-0078 от 20 ноября 2024.",
    "Оплата за консалтинговые услуги по проекту 'Эффективность' по счету №987-123 от 05.08.2024.",
    "За поставку строительных материалов для проекта 'Новострой' по договору №506-ТХ345 от 01 сентября 2023.",
    "Оплата за Аренду спецтехники по договору №34611 от 15 апреля 2024, адрес: Москва.",
    "Оплата за юридические услуги по делу №Ю16034 от 18 августа 2024.",
    "Услуги переводчика для встречи по договору №ТР-0021 от 25 февраля 2024.",
    "Техническая поддержка и обновление ПО по договору №П2024-88 от 14 октября 2024.",
    "Оплата за проведение онлайн-конференции по счету №КОНФ-5577 от 03.11.2024.",
    "Услуги по разработке маркетинговой стратегии по договору №МАРК-4786 от 28 января 2024.",
    "Оплата за складские услуги по контракту №СК-2025 от 10 сентября 2024.",
    "Оплата за контроль качества товаров на складе по договору №КК-0456 от 11 апреля 2024.",
    "Проведение тренинга по финансовому анализу по договору №ФИН-987 от 19.05.2024.",
    "Услуги по управлению проектами по договору №ПРОЕКТ-1123 от 05 июня 2024.",
    "Техническое обслуживание офисной техники по контракту №ТО-9954 от 31 июля 2024.",
    "Оплата за профилактическое медобслуживание сотрудников по договору №МЕД-445 от 20.09.2024."
],
"FOOD_GOODS": 
[
    'Оплата за Сыр Российский порционный 300г по счету 12345678901234567890 от 12.03.2024г Сумма 840.00',
    'Оплата по товарной накладной 0281 от 15 января 2023 за макароны "Барилла".Сумма 1250-00 В т.ч. НДС 20% 208-33',
    'Оплата за Чай липтон черный 25 пакетиков по счету 09876543210987654321 от 05.08.2023г Сумма 230.00',
    'Оплата за Консервы рыбные "Килька в томате" по счету 87654321098765432109 от 11.09.2024г Сумма 300.00',
    'Оплата за Оливковое масло Extra Virgin 500мл по счету 13579246801357924680 от 07.11.2023г Сумма 940.00',
    'Оплата по товарной накладной 0864 от 20 февраля 2024 за хлеб "Бородинский".Сумма 150-00 В т.ч. НДС 20% 25-00',
    'Оплата за Шоколад 70% какао 100г по счету 24681357902468135790 от 03.05.2023г Сумма 280.00',
    'Оплата за Йогурт ежевичный 180г по счету 11223344556677889900 от 06.06.2024г Сумма 120.00',
    'Оплата за Вода минеральная "Ессентуки №4" 1,5л по счету 33445566778899001122 от 09.07.2023г Сумма 150.00',
    'Оплата за Сахар-песок 1кг по счету 44556677889900112233 от 22.04.2023г Сумма 75.00',
    'Оплата по товарной накладной 0793 от 17 марта 2024 за кофе молотый "Якобс".Сумма 1020-00 В т.ч. НДС 20% 170-00',
    'Оплата за Гречка 800г по счету 55667788990011223344 от 01.12.2023г Сумма 220.00',
    'Оплата за Сок апельсиновый 1л по счету 66778899001122334455 от 14.08.2023г Сумма 180.00',
    'Оплата за Кефир 1% 1л по счету 77889900112233445566 от 19.09.2024г Сумма 60.00',
    'Оплата за Конфеты "Рафаэлло" 150г по счету 88990011223344556677 от 28.10.2023г Сумма 490.00',
    'Оплата по товарной накладной 0907 от 25 апреля 2024 за печенье "Юбилейное".Сумма 360-00 В т.ч. НДС 20% 60-00',
    'Оплата за Мясо куриное охлажденное 1кг по счету 99001122334455667788 от 16.03.2023г Сумма 280.00',
    'Оплата за Молоко 3,2% 1л по счету 00112233445566778899 от 30.07.2023г Сумма 70.00',
    'Оплата за Макароны "Спагетти" 500г по счету 11223344556677889911 от 18.02.2024г Сумма 110.00',
    'Оплата по товарной накладной 0333 от 29 июня 2024 за масло сливочное 100г.Сумма 98-00 В т.ч. НДС 20% 16-33'
],
"NON_FOOD_GOODS": [
    "Оплата за Уход за одеждой и обувью по счету 11837472833255495630 от 21.06.2024г. Сумма 4110-00",
    "Счет на оплату № 0112 от 15 февраля 2024гг. Тетради общие. Сумма 1500.00 В т.ч. НДС(20%) 250.0",
    "Оплата за Лампы светодиодные Galaxy 11W по счету 60875498231001827465 от 2023.05.20 Сумма 2890-00",
    "Оплата за Стеклоочистители WaterClear, 2шт по счету 17382938401924567981 от 10/09/2026 Сумма 340-00",
    "Счет на оплату № 0034 от 01.04.2025гг. Пылесосы Bosch 2200W. Сумма 18900-00 В т.ч. НДС(20%) 3150-00",
    "Оплата за Удлинитель электрический 5м по счету 81930481279023471892 от 16 ноября 2025 Сумма 670-00",
    "Приобретение Канцелярский клей ПВА, 1кг по счету 39028471092845716372 от 26/07/2024 Сумма 870.00",
    "Аванс по Договору №47Б-12121 за Стальной кабель 50м. Сумма 112000,00 в т.ч. НДС 18666,67",
    "Оплата за Батарейки Duracell AA 12шт по счету 58301928372048910754 от 03/09/2023 Сумма 1390.00",
    "Счет на оплату № 0845 от 20 августа 2024гг. Контейнеры для хранения 10л. Сумма 5500.00 В т.ч. НДС(20%) 916.67",
    "Оплата за Вешалки для одежды пластиковые по счету 73820194729038465271 от 27/12/2023 Сумма 810-00",
    "Счет на оплату № 0193 от 07 марта 2022гг. Зонт от солнца. Сумма 9600.00 В т.ч. НДС(20%) 1600.0",
    "Оплата за Ортопедические подушки SoftSleep по счету 42985423019837569824 от 14/05/2025 Сумма 4850-00",
    "Приобретение Скатерть водоотталкивающая по счету 56920138409182742571 от 06/10/2026 Сумма 1230.00",
    "Счет на оплату № 0764 от 29 июля 2023гг. Фены для волос Rowenta. Сумма 14500-00 В т.ч. НДС(20%) 2416-67",
    "Оплата за Рамы для картин 50x70см по счету 30297471092845726381 от 11/07/2026 Сумма 2140-00",
    "Оплата за Кастрюли эмалированные 3л по счету 17483948209384757192 от 17 сентября 2023 Сумма 1600-00",
    "Счет на оплату № 0657 от 12.11.2025гг. Пледы шерстяные 150x200см. Сумма 7600-00 В т.ч. НДС(20%) 1266-67",
    "Аванс по Договору №32В-11012 за Зеркала настенные 60x80см. Сумма 88000,00 в т.ч. НДС 14666,67",
    "Оплата за Ручки для мебели металлические по счету 91384750120938475832 от 15/08/2024 Сумма 3200.00",
    "Счет на оплату № 0510 от 17 июня 2026гг. Утюги Philips с парогенератором. Сумма 21600.00 В т.ч. НДС(20%) 3600.0"
],
"LEASING": [
    "Оплата по договору финансовой аренды №01234H от 15.03.2023, акт приема-передачи 879 от 20 декабря 2024. В т.ч. НДС(20%) 710000-00",
    "Возврат аванса по договору лизинга №565 от 01.07.2023 без НДС. Сумма возврата: 35000.00",
    "Заключительный платеж по договору лизинга №0023/98 от 05.09.2023г., акт приема-передачи 1025 от 17/11/2024г. В т.ч. НДС(20%) 912345.67",
    "Возмещение затрат на регистрацию ТС по договору лизинга №188 от 22.04.2023, сумма 14500.00 руб, без НДС",
    "Дополнительная оплата по договору аренды №45678 от 12.02.2024, акт приема-передачи 1290 от 01.10.2024. В т.ч. НДС(20%) 98000-00",
    "Комиссионные расходы по договору лизинга №AB-001 от 01.01.2025. Сумма с НДС 20% 5600.00 руб.",
    "Частичный расчет по договору лизинга №6734 от 30.06.2023, акт 203 от 12/09/2024. В т.ч. НДС(20%) 675000-00",
    "Окончательный расчет по договору лизинга №99-0137 от 10.10.2023г., с актом от 22.11.2024. В т.ч. НДС 20% 405500-00",
    "Оплата за страховку по договору лизинга №920 от 18.02.2023, сумма 28000.00 без НДС",
    "Оплата по доп.соглашению к дог. лизинга №34567 от 04.05.2023, акт 789 от 18.10.2024. В т.ч. НДС(20%) 505000-00",
    "Возврат обеспечения по договору лизинга №998 от 07.07.2023 без НДС. Сумма: 15000.00",
    "Окончательное погашение по договору лизинга №77-045 от 23.03.2024. Акт приема-передачи 452 от 01/12/2024. В т.ч. НДС(20%) 504500-00",
    "Оплата транспортных расходов по договору аренды №AJ-550 от 12.11.2023, сумма без НДС 22500.00",
    "Взнос за страхование КАСКО по договору лизинга 562 от 22.06.2023г. Без НДС. Сумма: 40000.00",
    "Компенсация убытков по договору лизинга №0147U от 19.07.2023. Сумма: 32000.00, без НДС",
    "Итоговый расчет по договору №36-01112 от 14.05.2023г., акт 666 от 25.11.2024. В т.ч. НДС(20%) 815000-00",
    "Плата за договор лизинга №76-00456 от 03.01.2023. Сумма: 7250.00, в т.ч. НДС 20%",
    "Авансовый платеж по договору лизинга №02137 от 17.07.2023, акт 890 от 20.09.2024. В т.ч. НДС(20%) 644000-00",
    "Закрытие обязательств по договору лизинга №67-002 от 08.05.2023г., акт 432 от 15/11/2024г. В т.ч. НДС(20%) 732000-00"
],
"LOAN": [
    'Оплата по договору процентного займа №Д-00753 от 12.11.2024г. Без НДС',
    'Возврат денежных средств по договору займа №Е00125/45 от 15.10.2024г. Без НДС',
    'Предоставление денежных средств по договору займа №57Е-0932 от 04.09.2024. Без НДС',
    'Оплата по договору процентного займа (15%) №07Б-04561 от 21.02.2024г. Без НДС',
    'Возврат денежных средств по договору займа №Ф09832/23 от 10.03.2023г. Без НДС',
    'Предоставление денежных средств по договору займа №К-01987/40 от 05.01.2024. Без НДС',
    'Оплата по договору процентного займа (18%) №А-03214 от 29.04.2023г. Без НДС',
    'Возврат денежных средств по договору займа №Д04501-07 от 31.05.2024г. Без НДС',
    'Предоставление денежных средств по договору займа б/н от 09.08.2023. Без НДС',
    'Оплата по договору процентного займа №Ж-00678/22 от 17.07.2024г. Без НДС',
    'Возврат денежных средств по договору займа №С-05341/18 от 22.02.2024г. Без НДС',
    'Оплата по договору процентного займа (12%) №03В-01988 от 27.03.2024г. Без НДС',
    'Предоставление денежных средств по договору займа №Д-01122/56 от 08.12.2023. Без НДС',
    'Возврат денежных средств по договору займа №Ф-03751/45 от 19.05.2024. Без НДС',
    'Оплата по договору процентного займа (10%) №43-01871 от 25.06.2024г. Без НДС',
    'Предоставление денежных средств по договору займа №Ж00234/75 от 01.04.2023г. Без НДС',
    'Возврат денежных средств по договору займа №З-01467 от 23.10.2024г. Без НДС',
    'Оплата по договору процентного займа (22%) №44Е-01002/30 от 05 июля 2024г. Без НДС',
    'Предоставление денежных средств по договору займа №В09854-07 от 19.09.2023г. Без НДС',
    'Оплата по договору процентного займа №М-03345/80 от 06.01.2024г. Без НДС'
],
"REALE_STATE": 
[
    "Оплата по счету №45-02339Б от 12.03.2022г. аренда офиса в Москве по договору №98Б-0234 от 15.12.2021",
    "Доплата по счету №202339 от 20/06/2023 за аренду склада по договору №39458/ОВ от 10/03/2021",
    "Приобретение квартиры в Новосибирске, ул. Ленина, д.10, по договору №21-00456 от 01 сентября 2021",
    "Оплата по счету №98-55823 от 11/11/2023 аренда помещения июль 2024 г. по договору №123/Б от 22/03/2021",
    "Долевое инвестирование в строительство дома в г. Казань, ул. Цветочная, д.5, по договору №23-554 от 18/04/2023",
    "Заключение сделки по покупке офиса в г. Омск, ул. Советов, д.30 по договору №87-00322 от 02.02.2024",
    "Доплата за аренду офиса в Екатеринбурге по счету №45-00932 от 10/05/2022 по договору №42-2010 от 01.03.2020",
    "Покупка участка земли в г. Краснодар, ул. Полевая, д.15, договор №12-00487 от 06 мая 2022",
    "Оплата аренды торговой площади в ТЦ, г. Санкт-Петербург, по договору №76-0084B от 10/02/2022 по счету №12399 от 25/12/2023",
    "Заключение договора аренды офиса с видом на реку в г. Владивосток по договору №45/П от 02.01.2023",
    "Капитальные вложения в строительство жилого комплекса по адресу г. Уфа, ул. Солнечная, д.12 контракт №78/З от 05/09/2024",
    "Оплата первого взноса по договору аренды земельного участка в г. Курган, ул. Лесная, д.8 по счету №9876 от 09.03.2024",
    "Покупка жилой площади по адресу г. Волгоград, Новая ул., д.3, квартира №25, договор №65-12345 от 12.04.2025",
    "Заключение сделки по приобретению коммерческой недвижимости, г. Томск, ул. Кирова, д.5, договор №90-0031 от 07/07/2021",
    "Аренда помещения в Туле, ул. Октябрьская, по договору №56-9078 от 11/01/2023 по счету №10456 от 20/08/2023",
    "Приобретение офиса в г. Пермь, ул. Красная, д.1 по договору №32-4411E от 12.09.2022",
    "Инвестиции в проект по строительству многофункционального комплекса в г. Сочи, договор №21-MC0045 от 15 мая 2022",
    "Оплата задолженности по договору аренды торговой точки, г. Белгород, ТЦ 'Успех', по счету №332200 от 05/05/2024",
    "Заключение договора долевого строительства в г. Челябинск, ул. Дорожная, д.1, квартира 56, договор №44-00339 от 14/08/2023",
    "Приобретение гаража в г. Саратов, на ул. Гаражная, д.44, договор №75-FG003 от 08.06.2022"
],
"BANK_SERVICE": 
[
    "Оплата основного долга по Договору №Г-20145/23 от 12.11.2025 г. с ОАО 'Третий банк'",
    "Комиссионное вознаграждение за конвертацию валюты.",
    "Комиссия за услугу смс-информирования за июль 2024 года. НДС не облагается.",
    "Оплата процентов по кредиту с 01.10.2025 г. по 01.01.2026 г. по Договору №Д-00234/19",
    "Комиссия за ежемесячную выписку по счету. НДС не облагается.",
    "Комиссия за обработку исходящего перевода по системе SWIFT. НДС не облагается.",
    "Оплата основного долга и процентов по Договору №Г04123/28 от 05.04.2026 г.",
    "Начисление процентов с 15.05.2025 г. по 15.09.2025 г. по Договору №Г-00567/17",
    "Комиссия за восстановление доступа к интернет-банку. НДС не облагается.",
    "Комиссия за подтверждение платежа по требованию иностранного банка.",
    "Оплата задолженности по кредитной карте за период с 01.02.2025 г. по 28.02.2025 г.",
    "Комиссионное вознаграждение за страхование кредита по Договору №К05678/64",
    "Ежемесячная комиссия за доверительное управление активами. НДС не облагается.",
    "Комиссия за пересчет наличных денег. НДС не облагается.",
    "Оплата основного долга за кредит на недвижимость по Договору №Н00358/45",
    "Комиссия за услуги брокера по сделкам с ценными бумагами. НДС не облагается.",
    "Оплата за предоставление банковской гарантии по Договору №Г01928/64",
    "Комиссия за предоставление выписки по карточному счету за сентябрь 2026 г.",
    "Списание комиссии за хранение ценностей в индивидуальной ячейке. НДС не облагается.",
    "Комиссионное вознаграждение за открытие и ведение расчетного счета."
],
"NOT_CLASSIFIED": 
[
    "Оплата договора №83 от 07/24г. согласно счету 0734 от 28.07.2024г.",
    "Пополнение запасов для проекта, НДС не предусмотрен.",
    "Средства на личные нужды",
    "Перевод с одного счета на другой в рамках учетной политики",
    "Финансирование текущих операций, налог не начисляется",
    "Капиталовложение на собственные нужды",
    "Перевод на другой счет",
    "Инвестиция в собственные проекты",
    "Распределение средств между счетами",
    "Оплата по счету 9812 от 04.05.2024",
    "Возврат личных вложений",
    "Оплата по договору №123 от 12/23г. по счету 5678 от 15.01.2024г.",
    "Оплата дог 40.051059-ДА от за 09/24г.согл.сч 0311",
    "Возмещение понесенных расходов",
    "Пополнение фонда для экстренных нужд",
    "Перевод на депозит",
    "Оплата по счету 4589 от 10.10.2023, сумма 15000, в том числе НДС(20%) 2500",
    "Пополнение счета для личных нужд",
    "Погашение задолженности перед собой",
    "Средства от реализации активов"
], 

"TAX":
[
    'Оплата налогового штрафа',
    'Уплата пошлины',
    'Алиментные выплаты',
    'Перевод на алименты',
    'Налоговые отчисления',
    'Оплата транспортного налога',
    'Платеж гос. пошлина',
    'Выплата детских алиментов',
    'Пошлина за регистрацию',
    'Оплата государственной услуги',
    'Консульский взнос',
    'Уплата налогов',
    'Государственная госпошлина',
    'Штраф по налогам',
    'Оплата учетного сбора',
    'Внесение консульского сбора',
    'Уплата административного штрафа',
    'Алименты на детей',
    'Задолженность по налогам',
    'Оплата штрафа за нарушение'
]




}

In [29]:
data = []
for category, texts in df_dev_json.items():
    for text in texts:
        data.append({'text': text, 'class': category})

In [30]:
df_dev = pd.DataFrame(data)

In [31]:
df_res[df_res["class"]=="TAX"]["text"].to_list()

['Оплата штрафа',
 'государственная пошлина',
 'Оплата гос. услуги',
 'гос. пошлина',
 'Алименты',
 'Оплата налогов',
 'Оплата гос. пошлины',
 'Выплата на содержание родных детей',
 'Оплата налога',
 'гос.услуга',
 'Консульские сборы',
 'налоги',
 'Оплата государственной пошлины',
 'штраф',
 'Консульский сбор',
 'наолги',
 'Выплата на содеражние родных детей',
 'Оплата гос. услгуи',
 'Оплата штарфа']

In [32]:
df_res["class"].value_counts()

FOOD_GOODS        922
NON_FOOD_GOODS    895
SERVICE           884
LEASING           380
LOAN              380
REALE_STATE       256
BANK_SERVICE      215
NOT_CLASSIFIED    117
TAX                19
Name: class, dtype: int64

In [26]:
def get_text_data(dataframe, file_name="train.csv"):
    texts, labels = [], []
    for i, row in dataframe.iterrows():
        texts.append(row["text"])
        labels.append(row["class"])
    # print(texts)
    # Объединяем списки в записи
    data = zip(texts, labels)
    # Сохраняем в CSV файл с разделителем \t
    with open(f'data/flair_data/{file_name}', 'w', newline='') as file:
        writer = csv.writer(file, delimiter='\t')
        writer.writerows(data)
        

In [37]:
get_text_data(df_res, file_name="train.csv")

In [32]:
get_text_data(df_dev, file_name="dev.csv")